<a href="https://colab.research.google.com/github/IT-17005/Advanced-Cryptography/blob/main/One_time_Pad.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import os
import json
import base64
from typing import Tuple, Optional
from pathlib import Path
import secrets
import datetime

KEY_META_SUFFIX = ".meta.json"

def generate_key(length: int) -> bytes:
    """Generate length bytes of key using the OS CSPRNG (os.urandom / secrets.token_bytes)."""
    if length <= 0:
        raise ValueError("length must be positive")
    return secrets.token_bytes(length)

def encrypt_bytes(plaintext: bytes, key: bytes) -> bytes:
    """Encrypt bytes with key using XOR. key must be at least len(plaintext)."""
    if len(key) < len(plaintext):
        raise ValueError("key must be at least as long as plaintext for OTP")
    # XOR
    return bytes(p ^ k for p, k in zip(plaintext, key))

def decrypt_bytes(ciphertext: bytes, key: bytes) -> bytes:
    """Decrypt bytes with key (same as encrypt)."""
    return encrypt_bytes(ciphertext, key)  # symmetric xor

def encrypt_text(text: str, key: bytes, *, encoding='utf-8') -> str:
    """Encrypt text, return base64-encoded ciphertext string."""
    pt = text.encode(encoding)
    ct = encrypt_bytes(pt, key)
    return base64.b64encode(ct).decode('ascii')

def decrypt_text(b64cipher: str, key: bytes, *, encoding='utf-8') -> str:
    ct = base64.b64decode(b64cipher)
    pt = decrypt_bytes(ct, key)
    return pt.decode(encoding)

def save_key_file(path: str, key: bytes, meta: Optional[dict] = None) -> None:
    """Save key to path (binary) and metadata JSON to path + KEY_META_SUFFIX."""
    p = Path(path)
    # Ensure the parent directory exists
    p.parent.mkdir(parents=True, exist_ok=True)
    if p.exists():
        raise FileExistsError(f"Key file already exists: {path}")
    p.write_bytes(key)
    m = meta.copy() if meta else {}
    m.setdefault('length', len(key))
    m.setdefault('used', False)
    m.setdefault('created_by', 'one_time_pad.py')
    m.setdefault('notes', '')
    meta_path = str(p) + KEY_META_SUFFIX
    Path(meta_path).write_text(json.dumps(m, indent=2))

def load_key_file(path: str) -> Tuple[bytes, dict]:
    p = Path(path)
    if not p.exists():
        raise FileNotFoundError(path)
    key = p.read_bytes()
    meta_path = str(p) + KEY_META_SUFFIX
    meta = {}
    if Path(meta_path).exists():
        meta = json.loads(Path(meta_path).read_text())
    return key, meta

def mark_key_used(path: str, overwrite_meta: Optional[dict] = None) -> None:
    """Mark key as used in metadata. Does NOT delete the key file automatically."""
    p = Path(path)
    meta_path = str(p) + KEY_META_SUFFIX
    meta = {}
    if Path(meta_path).exists():
        meta = json.loads(Path(meta_path).read_text())
    meta['used'] = True
    meta['used_at_utc'] = datetime.datetime.now(datetime.UTC).isoformat()
    if overwrite_meta:
        meta.update(overwrite_meta)
    Path(meta_path).write_text(json.dumps(meta, indent=2))

def secure_erase(b: bytearray):
    """Attempt to overwrite a mutable bytearray in-place. Best-effort only."""
    for i in range(len(b)):
        b[i] = 0
    # attempt additional overwrites
    for i in range(len(b)):
        b[i] = 0xFF
    for i in range(len(b)):
        b[i] = 0

def example_usage():
    print('--- One-Time Pad example usage ---')
    msg = 'This is a secret message. Use OTP carefully!'
    print('Plaintext:', msg)
    pt = msg.encode('utf-8')
    key = generate_key(len(pt))

    key_file_path = '/content/otp_key.bin'
    meta_file_path = key_file_path + KEY_META_SUFFIX

    # Clean up from previous runs if files exist
    if Path(key_file_path).exists():
        os.remove(key_file_path)
    if Path(meta_file_path).exists():
        os.remove(meta_file_path)

    # save key to file (demo)
    save_key_file(key_file_path, key, meta={'desc': 'demo key, do not use for real secrets'})
    cipher_b64 = encrypt_text(msg, key)
    print('Ciphertext (base64):', cipher_b64[:60] + '...')
    # decrypt
    key2, meta = load_key_file(key_file_path)
    plaintext = decrypt_text(cipher_b64, key2)
    print('Recovered plaintext:', plaintext)
    # mark key used to avoid reuse
    mark_key_used(key_file_path)

if __name__ == '__main__':
    example_usage()


--- One-Time Pad example usage ---
Plaintext: This is a secret message. Use OTP carefully!
Ciphertext (base64): AsQ9lhzg/NS7kUdRh31lz5ZIb607X1zM6Fqq+EDjfmPa/1rWig3y4a/uqTs=...
Recovered plaintext: This is a secret message. Use OTP carefully!
